In [1]:
import os
os.chdir("../..")

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import (
    plot_training, 
    plot_confusion_matrix, 
    plot_precision_recall, 
    plot_roc_curve
)

C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


In [4]:
# Model Name (Change this to test other models)
model_name = "mobilenet_v2"

In [5]:
# Directories
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/plots", exist_ok=True)

In [6]:
# Load dataset with augmentation
print("[INFO] Loading augmented datasets...")
data_dir = "wildfire_dataset_scaled"
batch_size = 32
train_loader, val_loader, test_loader = load_datasets(
    data_dir, batch_size=batch_size, augmentation="augmented"
)
print("[INFO] Datasets loaded successfully!")


[INFO] Loading augmented datasets...
[INFO] Datasets loaded successfully!


C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\pydantic\main.py:214: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [9]:
def objective(trial):
    print(f"[DEBUG] Starting trial {trial.number}...")

    # Hyperparameter suggestions
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    dropout_prob = trial.suggest_float("dropout_prob", 0.2, 0.5)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])

    print(f"[DEBUG] Suggested hyperparameters: lr={lr}, weight_decay={weight_decay}, optimizer={optimizer_name}")

    # Initialize model
    model = initialize_model(
        model_name=model_name,
        num_classes=2,
        pretrained=True,
        freeze_all=False,
        dropout_prob=dropout_prob
    )
    model.to(device)

    # Optimizer setup
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model
    print("[INFO] Starting model training...")
    save_path = f"outputs/models/{model_name}_trial_best.pth"
    history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        save_path=save_path,
        early_stop_patience=5,
        monitor_metric="val_recall",
    )
    print("[INFO] Training completed!")

    # Return validation recall
    val_recall = history["val_recall"][-1]
    print(f"[INFO] Trial {trial.number} - Final Validation Recall: {val_recall:.4f}")
    return val_recall


In [ ]:
# Run Optuna Hyperparameter Tuning
print("[INFO] Running hyperparameter optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print(f"[INFO] Best parameters: {study.best_params}")

# Train the final model with the best parameters
print("[INFO] Training final model with best hyperparameters...")
final_model = initialize_model(
    model_name=model_name,
    num_classes=2,
    pretrained=True,
    freeze_all=False,  # All layers unfrozen
    dropout_prob=study.best_params["dropout_prob"]
)
final_model.to(device)

[I 2024-12-18 23:16:21,526] A new study created in memory with name: no-name-e72efb15-687f-47ed-b9f2-8e1e11ab876a
C:\Users\Yash\AppData\Local\Temp\ipykernel_14772\967085688.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
C:\Users\Yash\AppData\Local\Temp\ipykernel_14772\967085688.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)
C:\Users\Yash\AppData\Local\Temp\ipykernel_14772\967085688.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. U

[INFO] Running hyperparameter optimization...
[DEBUG] Starting trial 0...
[INFO] Training the model...

Starting training...



[INFO] Learning rate adjusted to: 0.000134
[INFO] Best model saved with val_recall: 0.7480
Epoch [1]: Train Loss: 0.5548, Train Acc: 0.7107 | Val Loss: 0.4814, Val Acc: 0.7413, Val Recall: 0.7480, Val F1: 0.7797


[INFO] Learning rate adjusted to: 0.000134
[INFO] Best model saved with val_recall: 0.8984
Epoch [2]: Train Loss: 0.4022, Train Acc: 0.8018 | Val Loss: 0.4497, Val Acc: 0.7886, Val Recall: 0.8984, Val F1: 0.8387


[INFO] Learning rate adjusted to: 0.000134
[INFO] No improvement in val_recall. Patience: 1/5
Epoch [3]: Train Loss: 0.3595, Train Acc: 0.8336 | Val Loss: 0.3577, Val Acc: 0.8507, Val Recall: 0.8902, Val F1: 0.8795


...

[INFO] Learning rate adjusted to: 0.000079
[INFO] No improvement in val_recall. Patience: 1/5
Epoch [3]: Train Loss: 0.6530, Train Acc: 0.6301 | Val Loss: 0.6435, Val Acc: 0.6294, Val Recall: 0.9431, Val F1: 0.7569


[INFO] Learning rate adjusted to: 0.000079
[INFO] Best model saved with val_recall: 0.9756
Epoch [4]: Train Loss: 0.6534, Train Acc: 0.6206 | Val Loss: 0.6295, Val Acc: 0.6493, Val Recall: 0.9756, Val F1: 0.7729


[INFO] Learning rate adjusted to: 0.000079
[INFO] No improvement in val_recall. Patience: 1/5
Epoch [5]: Train Loss: 0.6342, Train Acc: 0.6449 | Val Loss: 0.6357, Val Acc: 0.6468, Val Recall: 0.9715, Val F1: 0.7710


[INFO] Learning rate adjusted to: 0.000079
[INFO] No improvement in val_recall. Patience: 2/5
Epoch [6]: Train Loss: 0.6295, Train Acc: 0.6513 | Val Loss: 0.6257, Val Acc: 0.6368, Val Recall: 0.9512, Val F1: 0.7622


Epoch [7] - Training:   0%|                                                                     | 0/59 [00:00<?, ?it/s]

In [ ]:
# Optimizer and Scheduler
optimizer = optim.Adam(
    final_model.parameters(), 
    lr=study.best_params["lr"], 
    weight_decay=study.best_params["weight_decay"]
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

# Loss Function
criterion = nn.CrossEntropyLoss()

In [ ]:
# Final Training
final_save_path = f"outputs/models/{model_name}_final.pth"
history = train_model(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    save_path=final_save_path,
    early_stop_patience=5,
    monitor_metric="val_recall",
    gradient_clip_val=1.0,  # Gradient Clipping
)

In [ ]:
# Final Evaluation and Visualization
print("[INFO] Evaluating final model...")
metrics = evaluate_model(final_model, test_loader, ["No Fire", "Fire"], device)

In [ ]:
# Plots
plot_training(history, f"outputs/plots/{model_name}_training_curve.png")
plot_confusion_matrix(
    metrics["confusion_matrix"], 
    ["No Fire", "Fire"], 
    output_path=f"outputs/plots/{model_name}_confusion_matrix.png"
)
if "y_true" in metrics and "y_probs" in metrics:
    y_true = metrics["y_true"]
    y_probs = [prob[1] for prob in metrics["y_probs"]]
    plot_precision_recall(y_true, y_probs, f"outputs/plots/{model_name}_precision_recall_curve.png")
    plot_roc_curve(y_true, y_probs, f"outputs/plots/{model_name}_roc_curve.png")

print("[INFO] Final results saved successfully!")